In [6]:
from preprocessing import *
import numpy as np
import matplotlib.pyplot as plt
import os
from tqdm import tqdm

In [7]:
parent_dir = './Data/In-lab/'
paths = sorted(list(os.listdir(parent_dir)))
data = []
for path in tqdm(paths):
    data.append(get_data(parent_dir+path))
data = np.concatenate(data, axis=0)

100%|██████████| 16/16 [00:06<00:00,  2.63it/s]


In [8]:
data.shape

(1509, 15016)